<a href="https://colab.research.google.com/github/Vvijayaragupathy-uno/Data-Science-EBooks/blob/main/Ollama_MMLU_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#@title #Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Thu Mar 13 13:00:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#@title # Setup
%cd /content/
!apt install lshw
!curl -fsSL https://ollama.com/install.sh | sh
!git clone --depth 1 https://github.com/chigkim/Ollama-MMLU-Pro
%cd Ollama-MMLU-Pro
!pip install -r requirements.txt


/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  pci.ids usb.ids
The following NEW packages will be installed:
  lshw pci.ids usb.ids
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 790 kB of archives.
After this operation, 2,988 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 790 kB in 1s (843 kB/s)
Selecting previously unselected package lshw.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../lshw_02.19.git.2021.06.19.996aaad9c7-2build1_amd64.deb ...
Unpacking lshw (02.19.git.2021.06.19.996aaad9c7-2build

In [3]:
#@title # Run server

%env OLLAMA_NUM_PARALLEL=16
%env OLLAMA_FLASH_ATTENTION=1

import threading
import subprocess
server = threading.Thread(target=lambda: subprocess.run(['ollama', 'serve'], check=True), daemon=True)
server.start()
import  time

time.sleep(2)

env: OLLAMA_NUM_PARALLEL=16
env: OLLAMA_FLASH_ATTENTION=1


In [4]:
#@title Download
print("Downloading model...")
!ollama pull llama3:8b-instruct-q8_0 &>/dev/null


In [5]:
#@title Run Test

!rm -rf eval_results
!python run_openai.py \
    --url http://localhost:11434/v1 \
    --model llama3:8b-instruct-q8_0 \
    --category 'computer science' \
    --verbosity 0 \
    --parallel 16


2025-03-13 12:22:05.366657
{
	"comment": "",
	"server": {
		"url": "http://localhost:11434/v1",
		"model": "llama3:8b-instruct-q8_0",
		"timeout": 600.0
	},
	"inference": {
		"temperature": 0.0,
		"top_p": 1.0,
		"max_tokens": 2048,
		"system_prompt": "The following are multiple choice questions (with answers) about {subject}. Think step by step and then finish your answer with \"the answer is (X)\" where X is the correct letter choice.",
		"style": "multi_chat"
	},
	"test": {
		"subset": 1.0,
		"parallel": 16
	},
	"log": {
		"verbosity": 0,
		"log_prompt": true
	}
}
README.md: 100% 10.9k/10.9k [00:00<00:00, 49.7MB/s]
test-00000-of-00001.parquet: 100% 4.15M/4.15M [00:00<00:00, 40.3MB/s]
validation-00000-of-00001.parquet: 100% 45.3k/45.3k [00:00<00:00, 115MB/s]
Generating test split: 100% 12032/12032 [00:00<00:00, 109982.40 examples/s]
Generating validation split: 100% 70/70 [00:00<00:00, 18279.25 examples/s]
assigned subjects ['computer science']
Testing computer science...
100% 410/41

In [11]:
file_path = "eval_results/llama3-8b-instruct-q8_0/report.txt"
with open(file_path, "r") as file:
    content = file.read()
print(content)

In [12]:
from google.colab import files
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
| overall | computer science |
| ------- | ---------------- |
| 39.76   | 39.76            |

SyntaxError: invalid syntax (<ipython-input-13-c8bb3e7cfc73>, line 1)

In [8]:
def evaluate(subjects):
    test_df, dev_df = load_mmlu_pro()
    if not subjects:
        subjects = list(test_df.keys())
    print("Assigned subjects:", subjects)
    lock = threading.Lock()
    system_prompt = config["inference"]["system_prompt"]
    wrong_answers = []  # List to store wrong answers

    for subject in subjects:
        start = time.time()
        print(f"Testing {subject}...")
        config["inference"]["system_prompt"] = system_prompt.replace("{subject}", subject)
        test_data = test_df[subject]
        output_res_path = os.path.join(output_dir, subject + "_result.json")
        output_summary_path = os.path.join(output_dir, subject + "_summary.json")
        res, category_record = update_result(output_res_path, lock)

        with ThreadPoolExecutor(max_workers=config["test"]["parallel"]) as executor:
            futures = {
                executor.submit(run_single_question, each, dev_df, res): each
                for each in test_data
            }
            for future in tqdm(
                as_completed(futures), total=len(futures), smoothing=0.0, ascii=True
            ):
                each = futures[future]
                label = each["answer"]
                category = subject
                prompt, response, pred, exist = future.result()
                if exist:
                    continue
                if response is not None:
                    res, category_record = update_result(output_res_path, lock)
                    if category not in category_record:
                        category_record[category] = {"corr": 0.0, "wrong": 0.0}
                    if config["log"]["log_prompt"]:
                        each["prompt"] = prompt
                    each["response"] = response
                    each["pred"] = pred
                    res.append(each)
                    if pred is not None:
                        if pred == label:
                            category_record[category]["corr"] += 1
                        else:
                            category_record[category]["wrong"] += 1
                            # Add wrong answer to the list
                            wrong_answers.append({
                                "category": category,
                                "question": each["question"],
                                "options": each["options"],
                                "pred": pred,
                                "answer": label,
                            })
                    else:
                        category_record[category]["wrong"] += 1
                    save_res(res, output_res_path, lock)
                    save_summary(category_record, output_summary_path, lock)
                    res, category_record = update_result(output_res_path, lock)
        save_res(res, output_res_path, lock)
        log(f"Finished testing {subject} in {elapsed(start)}.")
        save_summary(category_record, output_summary_path, lock, report=True)

    # Display wrong answers
    if wrong_answers:
        print("\nWrong Answers:")
        for i, wrong_answer in enumerate(wrong_answers, 1):
            print(f"\nWrong Answer {i}:")
            print(f"Category: {wrong_answer['category']}")
            print(f"Question: {wrong_answer['question']}")
            print(f"Options: {wrong_answer['options']}")
            print(f"Model Prediction: {wrong_answer['pred']}")
            print(f"Correct Answer: {wrong_answer['answer']}")
    else:
        print("\nNo wrong answers found.")

In [9]:
# Save wrong answers to a file
if wrong_answers:
    wrong_answers_path = os.path.join(output_dir, "wrong_answers.json")
    with open(wrong_answers_path, "w") as f:
        json.dump(wrong_answers, f, indent=4)
    print(f"\nWrong answers saved to: {wrong_answers_path}")

NameError: name 'wrong_answers' is not defined